# naver_movie_commot_classifier/nsmc.ipynb

네이버영화 댓글 감성분석

- https://github.com/e9t/nsmc

In [1]:
!uv pip install requests pandas scikit-learn 

Using Python 3.10.19 environment at: c:\Users\Playdata\Documents\SKN21\08_NLP-자연어처리\.venv4
Resolved 15 packages in 111ms
Prepared 3 packages in 1.61s
Installed 13 packages in 690ms
 + certifi==2025.11.12
 + charset-normalizer==3.4.4
 + idna==3.11
 + joblib==1.5.2
 + numpy==2.2.6
 + pandas==2.3.3
 + pytz==2025.2
 + requests==2.32.5
 + scikit-learn==1.7.2
 + scipy==1.15.3
 + threadpoolctl==3.6.0
 + tzdata==2025.2
 + urllib3==2.5.0


In [2]:
!uv pip install kiwipiepy

Using Python 3.10.19 environment at: c:\Users\Playdata\Documents\SKN21\08_NLP-자연어처리\.venv4
Resolved 5 packages in 102ms
Prepared 1 package in 196ms
Installed 3 packages in 31ms
 + kiwipiepy==0.22.1
 + kiwipiepy-model==0.22.0
 + tqdm==4.67.1


In [1]:
import requests
from io import StringIO
import pandas as pd
url = "https://raw.githubusercontent.com/e9t/nsmc/refs/heads/master/ratings.txt"
res = requests.get(url)
if res.status_code == 200:
    str_id = StringIO(res.text) # 메모리의 문자열(str)으로부터 값을 읽을 수 있는 InputStream
    df = pd.read_csv(str_id, sep="\t")


In [7]:
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [9]:
df['label'].value_counts()

label
1    100000
0    100000
Name: count, dtype: int64

In [10]:
df.isnull().sum()

id          0
document    8
label       0
dtype: int64

In [12]:
# 결측치 제거
df = df.dropna()
df.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [21]:
# 중복된 댓글(document) 확인
df[df.duplicated(subset=["document"], keep=False)].sort_values('document')
# keep : False - 모든 중복행들을 True, "first", "last" (중복된 것들 중 첫번째/마지막 것만 True)

,id,document,label


In [ ]:
# 중복된 댓글이 있는 행은 하나만 남기고 제거
df = df.drop_duplicates(subset='document')

In [22]:
df.shape

(194543, 3)

In [23]:
df.reset_index(drop=True, inplace=True)

In [6]:
!uv pip install kiwipiepy

Using Python 3.10.19 environment at: c:\Users\Playdata\Documents\SKN21\08_NLP-자연어처리\.venv4
Resolved 5 packages in 81ms
Installed 1 package in 33ms
 + kiwipiepy==0.22.1


In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev pyhon3-dev
pip3 install jpype1
pip3 install konlpy

Couldn't find program: 'bash'


In [13]:
!uv apt-get update

error: unrecognized subcommand 'apt-get'

Usage: uv [OPTIONS] <COMMAND>

For more information, try '--help'.


In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

In [1]:
# 네이버 댓글 토큰화 처리 클래스
# 1. 다운로드
# 2. 기본적인 전처리 진행
# 3. 형태소기반 토큰화 진행
import os
import requests
import pandas as pd
from kiwipiepy import Kiwi

class NSMCTokenizer:

    def __init__(self, save_path:str):
        # save_path : 데이터셋 다운 후 저장할 디렉토리 경로.
        self.kiwi = Kiwi(num_workers=-1) # num_works: 병렬처리 시 사용할 cpu 개수. -1 : 모두

        # nsmc 데이터셋을 loading - load_nsmc_dataset() 메소드 이용
        df = self.load_nsmc_dataset(save_path)

        # 전처리 + 형태소 기반 토큰화
        self.nsmc_df = self.preprecess(df)

    def load_nsmc_dataset(self, save_path:str="data")->pd.DataFrame:
        """
        NSMC 데이터셋을 다운받아서 저장 및 DataFrame으로 반환.
        Args:
            save_path: 데이터셋 csv파일을 저장할 디렉토리
        Returns:
            pd.DataFrame
                네이버 영화댓글 감성분석 Dataset.
                feature: id-댓글 ID, document: 댓글내용, label: Target(0:부정, 1:긍정)

        Raise:
            Exception: 파일을 다운받지 못하면 발생
        """
        # 디렉토리 생성
        os.makedirs(save_path, exist_ok=True)
        file_path = os.path.join(save_path, 'ratings.txt')

        try:
            df = pd.read_csv(file_path, sep='\t', encoding='UTF-8')
        except:
            #Exception 발생: csv파일이 없는 경우 -> 다운로드
            if os.path.exists(file_path): #True : 있는 파일/디렉토리, False:없다.
                os.remove(file_path)
                
            url = "https://raw.githubusercontent.com/e9t/nsmc/refs/heads/master/ratings.txt"
            res = requests.get(url)
            if res.status_code == 200:
                # file_path에 저장 후 DataFrame생성
                with open(file_path, 'wt', encoding='utf-8') as fw:
                    fw.write(res.text)

                df = pd.read_csv(file_path, sep='\t', encoding='utf-8')

            else:
                # 다운시 문제 발생
                raise Exception("csv 파일을 다운받지 못했습니다. status 코드:", res.status_code)
        return df
    def tokenize(self, doc: str) -> str:
        """
        개별 댓글을 받아서 공백 처리, 토큰화 처리한 결과를 다시 string으로 만들어서 반환.

        Args:
            doc: str-처리할 댓글 문서 1개
        Returns:
            str - 처리 결과
        """
        doc = self.kiwi.space(doc)
        token_list = []
        try:
            # 토큰화: 원형(lemma)을 저장.
            for token in self.kiwi.tokenize(doc):
                token_list.append(token.lemma)
        except:
            pass

        return ' '.join(token_list)
    
    def preprocess(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Dataset 전처리 + 토큰화 작업
        전처리 : 결측치 제거, 댓글 중복데이터(중복행) 삭제
        토큰화 : tokenize() 메소드를 이용해서 토큰화작업
        Args :
            df: pd.DataFrame - 전처리 대상 DataFrame

        Returns:
            pd.DataFrame - 전처리 결과
            
        """
        res_df = df.dropna() #결측치 처리
        res_df = res_df.drop_duplicates(subset="document") # 중복행 제거
        
        # 공백 교정, 토큰화 -> tokenize()
        res_df['document'] = res_df['document'].apply(self.tokenize)

        # document에 글자수가 0인 행들 제거

        res_df['document'].str.strip()

        drop_idx=res_df[res_df['document'].str.strip().str.len() == 0].index
        res_df = res_df.drop(index=drop_idx).reset_index(drop=True)
        return res_df
    

In [14]:
# 문자열 타입 Series.str.메소드 
# -> str accessor: Series의 원소들을 일괄처리.
# accessor: str(문자열처리), dt(일시-datatime타입 시리즈), plot: 시각화
a.nsmc_df['document'].str.strip().str.len()

NameError: name 'a' is not defined

In [2]:
import time
s = time.time()

nsmc = NSMCTokenizer("data")

print(time.time() - s, "초")

Exception: Cannot open extract.mdl for WordDetector

In [ ]:
nsmc.tokenize("이 영화는 정말 재미있다.")
nsmc.tokenize("이 시리즈는 전작보다 항상 더 잘나온다.")

NameError: name 'nsmc' is not defined

In [4]:
############모델링
# dataset 분리
df = nsmc.nsmc_df.copy()
X = df['document']
y = df['label']

X.shape, y.shape


NameError: name 'nsmc' is not defined

In [ ]:
# train/test/validation set으로 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train
)

y_train.shape, y_valid.shape, y_test.shape

In [ ]:
########################################################################
# 모델 생성
# pipleline: 전처리 - TfidfVectorizer, 모델: LogisticRegression
########################################################################
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

tiv = TfidfVectorizer()
model = LogisticRegression()
steps = [
    ("TF-IDF", tiv),
    ("model", model)
]
pipeline = Pipeline(steps=steps, verbose=True)

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
# pred = pipeline(X_valid)
# accuracy_score(pred, y_valid)
validation_score = pipeline.score(X_valid, y_valid)
# 기본 metrics를 계산. 분류:accuracy, 회귀:r2_score
validation_score

In [ ]:
train_score = pipeline.score(X_train, y_train)
train_score

In [ ]:
test_score = pipeline.score(X_test, y_test)
test_score

In [ ]:
tiv = pipeline.steps[0][1]
vocab = tiv.get_feature_names_out()
len(vocab)

In [ ]:
##################################
# 새로운 데이터 추론
##################################
def predict(pipeline, *comments):
    # 1. 기본 전처리 - nsmc.tokenize()
    # 2. 추론
    pre_comment = [nsmc.tokenize(comment) for comment in comments]
    pred = pipeline.predict(pre_comment)
    return pred

In [ ]:
result = predict(
    pipeline,
    "이영화 별로다.",
    "아무 생각 없이 봤는데 시간가는줄 모르고 봤다.",
    "배우들 연기가 끝내준다.",
    "내 소중한 시간이...."
)